# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrape Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

helper functions to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### Request the Falcon9 Launch Wiki page from its URL


In [5]:
response = requests.get(static_url)
response.status_code

200

##### Create a `BeautifulSoup` object from the response


In [6]:
soup = BeautifulSoup(response.text, 'lxml')

In [7]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### Extract all column/variable names from the HTML table header


In [8]:
html_tables = soup.find_all('table')

Starting from the third table, which is the target table containing the actual launch records.


In [9]:
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">

<tbody><tr>

<th scope="col">Flight No.

</th>

<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)

</th>

<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>

</th>

<th scope="col">Launch site

</th>

<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>

</th>

<th scope="col">Payload mass

</th>

<th scope="col">Orbit

</th>

<th scope="col">Customer

</th>

<th scope="col">Launch<br/>outcome

</th>

<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>

</th></tr>

<tr>

<th rowspan="2" scope="row" style="text-alig

##### Extract column name one by one


In [10]:
column_names = []

for row in first_launch_table.find_all('th'):
    name = extract_column_from_header(row)
    if name is not None and len(name)  > 0:
        column_names.append(name)


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


#### Create a data frame by parsing the launch HTML tables


In [12]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]
print(launch_dict)

{'Flight No.': [], 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}


##### Fill up the dictionary with launch records extracted from table rows


In [13]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        row=rows.find_all('td')
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            print(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print(payload)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            print(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else "None"
            launch_dict['Customer'].append(customer)
            print(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print(booster_landing)
            
print("extracted rows: ", extracted_row)

1

4 June 2010

18:45

F9 v1.0B0003.1

CCAFS

Dragon Spacecraft Qualification Unit

Dragon Spacecraft Qualification Unit

LEO

SpaceX

Success



Failure

2

8 December 2010

15:43

F9 v1.0B0004.1

CCAFS

Dragon

Dragon

LEO

NASA

Success

Failure

3

22 May 2012

07:44

F9 v1.0B0005.1

CCAFS

Dragon

Dragon

LEO

NASA

Success

No attempt



4

8 October 2012

00:35

F9 v1.0B0006.1

CCAFS

SpaceX CRS-1

SpaceX CRS-1

LEO

NASA

Success



No attempt

5

1 March 2013

15:10

F9 v1.0B0007.1

CCAFS

SpaceX CRS-2

SpaceX CRS-2

LEO

NASA

Success



No attempt



6

29 September 2013

16:00

F9 v1.1B1003

VAFB

CASSIOPE

CASSIOPE

Polar orbit

MDA

Success

Uncontrolled

7

3 December 2013

22:41

F9 v1.1

CCAFS

SES-8

SES-8

GTO

SES

Success

No attempt

8

6 January 2014

22:06

F9 v1.1

CCAFS

Thaicom 6

Thaicom 6

GTO

Thaicom

Success

No attempt

9

18 April 2014

19:25

F9 v1.1

Cape Canaveral

SpaceX CRS-3

SpaceX CRS-3

LEO

NASA

Success



Controlled

10

14 July 2014

15:15

In [14]:
for key in launch_dict:
    print(key, len(launch_dict[key]))

Flight No. 121

Launch site 121

Payload 121

Payload mass 121

Orbit 121

Customer 121

Launch outcome 121

Version Booster 121

Booster landing 121

Date 121

Time 121


##### Create a dataframe from the dictionary


In [15]:
df=pd.DataFrame(launch_dict)

##### Export it to a <b>CSV</b>


In [16]:
df.to_csv('spacex_web_scraped.csv', index=False)
